## Import all the libraries 

In [1]:
from pymongo import MongoClient
import re
from pymongo import ASCENDING, DESCENDING
import pandas as pd
from pandas.io.json import json_normalize
import requests



## Import the dataset Companies

In [2]:
dbName = "datamad0320"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/datamad0320


# First Analysis: Decide the City


## What is the city with more Gaming companies?

This could be interresting to see where to install the new office of our Gaming company.

## 1) Get ALL the offices with category Gaming

In [3]:
a = list(db.companies.aggregate([{"$unwind":"$offices"},{"$project":{"offices":1,"name":1,"category_code":1}}]))

In [4]:
df = pd.DataFrame(json_normalize(a))
print(df.shape)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


(16705, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN


## 2) Filter only Category Gaming that has the city information

In [5]:
df_gaming = df

df_gaming.drop(df_gaming[df_gaming.category_code != "games_video"].index, axis=0, inplace=True)


In [6]:
df_gaming.shape

(956, 12)

In [7]:
df_gaming.dropna(subset = ['offices.city'])
df_gaming.shape

(956, 12)

## Result: New York

In [8]:
df_gaming["offices.city"].value_counts()

New York         75
San Francisco    68
                 54
London           36
Los Angeles      31
                 ..
Santa Cruz        1
Malvern           1
Columbia          1
Flowermound       1
Venice            1
Name: offices.city, Length: 363, dtype: int64

## Designers requirement: City with design companies

Search for cities with design companies, because this will be one of our requirements for the new office. 

In [9]:
df = pd.DataFrame(json_normalize(a))
print(df.shape)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


(16705, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN


## Filter only Category Design/Web that has the city information

In [10]:
df_design = df

df_design.drop(df_design[(df_design['category_code'] != "web") & (df_design['category_code'] != "design")].index , inplace=True)

In [11]:
print(df_design.shape)
df_design.head()

(3085, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
5,52cdef7c4bab8bd675297d90,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
6,52cdef7c4bab8bd675297d91,Geni,web,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
7,52cdef7c4bab8bd675297d96,Gizmoz,web,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


In [12]:
df_design.dropna(subset = ['offices.city'])
df_design.shape

(3085, 12)

## Result: New York is the second city with more design companies

In [13]:
df_design["offices.city"].value_counts()

San Francisco    217
New York         193
                 184
London           113
Seattle           63
                ... 
Falkirk            1
Boynton Beach      1
Detroit            1
Lueneburg          1
Puyallup           1
Name: offices.city, Length: 900, dtype: int64

Now that we already have New York as our city, due to their number of gaming and design companies, we need to continue to investigate in which coordinate we will launch the new office.

# First Requirement: Nearby Design companies

Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

Designers represents 23% of the total employees of the company.

Filter all the DESIGN COMPANIES COORDINATES IN NEW YORK. Our new office needs to be nearby these range of coordinates.

In [39]:
all_offices = list(db.companies.find({"offices.city":{"$eq":"New York"}},{"offices":1,"name":1,"category_code":1}))


In [40]:
companydata = pd.DataFrame(all_offices) 
companydata.shape

(832, 4)

In [41]:
companydata = companydata.explode("offices")
companydata.head()

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '710 - 2nd Ave..."
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '270 Lafayette..."
1,52cdef7c4bab8bd675297d9d,Joost,games_video,"{'description': '', 'address1': '100 5th Ave F..."
2,52cdef7c4bab8bd675297d9f,Viacom,web,"{'description': 'HQ', 'address1': '1515 Broadw..."
3,52cdef7c4bab8bd675297da7,AddThis,advertising,"{'description': 'HQ - Virginia', 'address1': '..."


In [42]:
def officeToGeoPoint(row):
    office = row.offices
    if type(office) == dict:
        if 'latitude' in office and 'longitude' in office:
            if(type(office["latitude"])) == float and type(office["longitude"]) == float:
                return ({
                    "type":"Point",
                    "coordinates":[office["longitude"],office["latitude"]]
                },"success")
            else:
                return(None,"Invalid lat lat and long")
        else:
            return (None,"No lat and long keys in office dict")
    return (None,"No office")

In [43]:
cleaned_offices = companydata.apply(officeToGeoPoint,axis=1, result_type="expand")
cleaned_offices.columns = ["office","clean_state"]

cleaned_offices.head()

,office,clean_state
0,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
1,"{'type': 'Point', 'coordinates': [-74.0094471,...",success
2,"{'type': 'Point', 'coordinates': [-73.986011, ...",success
3,"{'type': 'Point', 'coordinates': [-77.245195, ...",success


In [44]:
company_processed = pd.concat([companydata,cleaned_offices], axis=1)

In [45]:
company_processed = company_processed[["name","category_code","office","clean_state"]]

## Filter only companies with coordinates

In [48]:
company_processed = company_processed[company_processed.clean_state == 'success']
company_processed.head()

,name,category_code,office,clean_state
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
1,Joost,games_video,"{'type': 'Point', 'coordinates': [-74.0094471,...",success
2,Viacom,web,"{'type': 'Point', 'coordinates': [-73.986011, ...",success
3,AddThis,advertising,"{'type': 'Point', 'coordinates': [-77.245195, ...",success


## Filter only Design/Web companies

In [49]:
company_processed = company_processed[company_processed.category_code == 'web']
company_processed.head()

,name,category_code,office,clean_state
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
2,Viacom,web,"{'type': 'Point', 'coordinates': [-73.986011, ...",success
5,Meetup,web,"{'type': 'Point', 'coordinates': [-73.995722, ...",success
12,Meebo,web,"{'type': 'Point', 'coordinates': [-122.079017,...",success


In [50]:
company_processed.shape

(179, 4)

We have a range of 179 coordinates in New York to use as a reference to search for the new office. 

## Convert into Json

In [51]:
company_processed.to_json("../Input/companies_clean.json",orient="records")

# Filtering the coordinates based on the requirements

## Geocoding

Gives the coordinates from an address.

In [52]:
def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}",params={"json":1})
    data = res.json()
    print(res)
    return {
    "type": "Point",
    "coordinates": [float(data["longt"]), float(data["latt"])]
  }

## Query Nearby

Filter the coordinates that are in the requested radius from a specific point of reference.

In [53]:
def getOfficeNear(address, maxDist=1000):
    point = geocode(address)
    return {
       "office": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       }
    }

## Specifing the point of reference

In [57]:
def pointofReference(point):
    query = getOfficeNear(point)
    return query

In [59]:
pointofReference("Times Square New York")

<Response [200]>


{'office': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-73.98964, 40.75677]},
   '$maxDistance': 1000}}}

## Result

In [55]:
cur = db.companies_prepared.find(query, {"_id":0})
print(cur.count())
result = list(cur)
result

183


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[{'name': 'Real Time Content',
  'category_code': 'advertising',
  'office': {'type': 'Point', 'coordinates': [-73.990286, 40.755959]},
  'clean_state': 'success'},
 {'name': 'ideeli',
  'category_code': 'ecommerce',
  'office': {'type': 'Point', 'coordinates': [-73.990396, 40.755978]},
  'clean_state': 'success'},
 {'name': 'Magnetic',
  'category_code': 'advertising',
  'office': {'type': 'Point', 'coordinates': [-73.990152, 40.758343]},
  'clean_state': 'success'},
 {'name': 'MindSmack',
  'category_code': 'games_video',
  'office': {'type': 'Point', 'coordinates': [-73.989987, 40.758492]},
  'clean_state': 'success'},
 {'name': 'MYSTYLEPOST',
  'category_code': 'network_hosting',
  'office': {'type': 'Point', 'coordinates': [-73.989622, 40.754901]},
  'clean_state': 'success'},
 {'name': 'Thomson Reuters',
  'category_code': 'public_relations',
  'office': {'type': 'Point', 'coordinates': [-73.9871847, 40.7564318]},
  'clean_state': 'success'},
 {'name': 'Thomson Reuters',
  'categ

## Get the final coordinates

In [60]:
def easyLatLng(row):
    of = row.office
    return {
        "latitude":of["coordinates"][1],
        "longitude":of["coordinates"][0]
    }

# https://api.mongodb.com/python/current/api/bson/objectid.html
df = pd.DataFrame(result)

df = pd.concat([df, df.apply(easyLatLng, axis=1, result_type="expand")], axis=1)
df

,name,category_code,office,clean_state,latitude,longitude
0,Real Time Content,advertising,"{'type': 'Point', 'coordinates': [-73.990286, ...",success,40.755959,-73.990286
1,ideeli,ecommerce,"{'type': 'Point', 'coordinates': [-73.990396, ...",success,40.755978,-73.990396
2,Magnetic,advertising,"{'type': 'Point', 'coordinates': [-73.990152, ...",success,40.758343,-73.990152
3,MindSmack,games_video,"{'type': 'Point', 'coordinates': [-73.989987, ...",success,40.758492,-73.989987
4,MYSTYLEPOST,network_hosting,"{'type': 'Point', 'coordinates': [-73.989622, ...",success,40.754901,-73.989622
...,...,...,...,...,...,...
178,Aleri,software,"{'type': 'Point', 'coordinates': [-73.9783534,...",success,40.759219,-73.978353
179,Warner Music Group,other,"{'type': 'Point', 'coordinates': [-73.9783534,...",success,40.759219,-73.978353
180,SheZoom,network_hosting,"{'type': 'Point', 'coordinates': [-73.9785086,...",success,40.753864,-73.978509
181,Social Median,web,"{'type': 'Point', 'coordinates': [-73.982575, ...",success,40.749630,-73.982575
